## Extraction of Data from iSDA Datasets (GEE) ##

In [35]:
# Import Google Earth Engine API and Initialize it. 
import ee

ee.Authenticate()
ee.Initialize(project="ey-data-and-ai-challenge")


In [9]:
# Other import statements to be added as needed

import pandas as pd
import numpy as np

In [23]:
# Read coordinates from water quality training dataset, drop given features and date since iSDA datasets are static, i.e. not date dependent.

wq_df = pd.read_csv('../data/water_quality_training_dataset.csv')
wq_df = wq_df.drop(columns=['Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus', 'Sample Date'])
wq_df['id'] = wq_df.index
wq_df.head()


,Latitude,Longitude,id
0,-28.760833,17.730278,0
1,-26.861111,28.884722,1
2,-26.450000,28.085833,2
3,-27.671111,27.236944,3
4,-27.356667,27.286389,4


In [29]:
# Convert Coordintaes to ee.Features

features = []

for index, row in wq_df.iterrows():
    feat = ee.Feature(
        ee.Geometry.Point([row['Longitude'], row['Latitude']]).buffer(100), #add a 100m buffer in case of inexact coordinates
        {'id': row['id']}
    )
    features.append(feat)

Read Google Earth Engine iSDA datasets (image) for extraction with specified coordinate bounds

In [30]:
# Reading all wanted iSDA datasets as EE images into a single image using ee.image.cat for quicker processing.

pH_img = ee.Image("ISDASOIL/Africa/v1/ph").select('mean_0_20').rename('pH')
phosphorous_img = ee.Image("ISDASOIL/Africa/v1/phosphorus_extractable").select('mean_0_20').rename('phosphorous')
cec_img = ee.Image("ISDASOIL/Africa/v1/cation_exchange_capacity").select('mean_0_20').rename('cec') # Cation Exchange Capacity
clay_img = ee.Image("ISDASOIL/Africa/v1/clay_content").select('mean_0_20').rename('clay') # clay content

# concatenate all images into a single image to query coordinates with

soil_img = ee.Image.cat([pH_img, phosphorous_img, cec_img, clay_img])



Create a Feature Collection from coordinates specified in wq_df and combined images in soil_img

In [33]:
fc = ee.FeatureCollection(features)
fc.size().getInfo()

9319

Use ee.reduceRegions() to query data from coordinates, averaging values taken over bounding buffer area

In [34]:
# Run geospatial data query using reduceRegions()

reducer = ee.Reducer.mean().combine(ee.Reducer.median(), sharedInputs=True)

soils_collection = soil_img.reduceRegions(collection=fc, reducer=reducer, scale = 30)

In [36]:
# Process data and export to Google Drive

task = ee.batch.Export.table.toDrive(
    collection=soils_collection,
    description="iSDA_csv_export",
    fileNamePrefix= "iSDA_features_training",
    fileFormat='CSV'
)
task.start()

### Mapping queried data back to water_quality_training coordinates and saving to csv

In [44]:
isda_df = pd.read_csv("../data/iSDA_features_training.csv")

# Drop irrelevant columns
isda_df.drop(columns=[".geo", "system:index"], inplace=True)

isda_df = isda_df.merge(wq_df, on='id', how='left')
isda_df.drop(columns=['id'], inplace=True)
isda_df

,cec_mean,cec_median,clay_mean,clay_median,pH_mean,pH_median,phosphorous_mean,phosphorous_median,Latitude,Longitude
0,24.201976,24.0,21.148754,21.0,75.285714,75.0,22.926434,23.0,-28.760833,17.730278
1,27.256276,27.0,29.204172,29.0,65.133646,65.0,21.057342,21.0,-26.861111,28.884722
2,23.430807,23.0,28.690019,28.0,64.278137,64.0,20.680648,21.0,-26.450000,28.085833
3,22.639576,23.0,25.220847,26.0,65.572103,65.0,22.007566,22.0,-27.671111,27.236944
4,25.096249,25.0,27.768108,28.0,63.993375,64.0,20.045393,20.0,-27.356667,27.286389
...,...,...,...,...,...,...,...,...,...,...
9314,21.786735,22.0,25.580164,25.0,55.673273,56.0,24.732548,25.0,-27.527500,30.858056
9315,27.256276,27.0,29.204172,29.0,65.133646,65.0,21.057342,21.0,-26.861111,28.884722
9316,25.324860,25.0,25.549216,26.0,64.699207,64.0,18.940779,19.0,-26.984722,26.632278
9317,23.971461,24.0,23.775857,24.0,69.031109,70.0,19.922538,20.0,-27.935000,26.126667


In [45]:
isda_df.to_csv("../data/iSDA_features_training.csv")

# Repeat for Validation Set

In [49]:
val_df = pd.read_csv("../data/submission_template.csv")
val_df['id'] = val_df.index
val_df

,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus,id
0,-32.043333,27.822778,01-09-2014,NaN,NaN,NaN,0
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN,1
2,-32.991639,27.640028,07-05-2015,NaN,NaN,NaN,2
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN,3
4,-32.000556,28.581667,01-10-2014,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...
195,-33.771111,25.386667,06-12-2012,NaN,NaN,NaN,195
196,-33.185361,27.390750,04-09-2014,NaN,NaN,NaN,196
197,-32.043333,27.822778,28-09-2015,NaN,NaN,NaN,197
198,-33.001667,25.161389,08-01-2015,NaN,NaN,NaN,198


In [50]:
features_val = []

for index, row in val_df.iterrows():
    feat_val = ee.Feature(
        ee.Geometry.Point([row['Longitude'], row['Latitude']]).buffer(100), #add a 100m buffer in case of inexact coordinates
        {'id': row['id']}
    )
    features_val.append(feat_val)

In [52]:
fc_val = ee.FeatureCollection(features_val)
soils_collection_val = soil_img.reduceRegions(collection=fc_val, reducer=reducer, scale = 30)

In [53]:
task_val = ee.batch.Export.table.toDrive(
    collection=soils_collection_val,
    description="iSDA_csv_export",
    fileNamePrefix= "iSDA_features_validation",
    fileFormat='CSV'
)
task_val.start()

In [54]:
isda_val_df = pd.read_csv("../data/iSDA_features_validation.csv")

# Drop irrelevant columns
isda_val_df.drop(columns=[".geo", "system:index"], inplace=True)

isda_val_df = isda_val_df.merge(val_df, on='id', how='left')
isda_val_df.drop(columns=['id'], inplace=True)
isda_val_df

,cec_mean,cec_median,clay_mean,clay_median,pH_mean,pH_median,phosphorous_mean,phosphorous_median,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,26.179883,26.0,26.132974,27.0,66.687388,67.0,25.588024,26.0,-32.043333,27.822778,01-09-2014,NaN,NaN,NaN
1,22.789987,23.0,25.544245,25.0,65.256120,65.0,25.510021,26.0,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN
2,23.254072,23.0,23.668477,24.0,60.640929,61.0,25.400431,25.0,-32.991639,27.640028,07-05-2015,NaN,NaN,NaN
3,18.426581,19.0,24.248131,24.0,58.755061,59.0,24.235986,24.0,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN
4,26.664381,27.0,29.381570,29.0,62.052236,62.0,26.756774,27.0,-32.000556,28.581667,01-10-2014,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,24.166771,24.0,25.327581,25.0,65.664889,66.0,26.233448,26.0,-33.771111,25.386667,06-12-2012,NaN,NaN,NaN
196,23.170123,23.0,25.235916,25.0,62.351212,62.0,26.665554,27.0,-33.185361,27.390750,04-09-2014,NaN,NaN,NaN
197,26.179883,26.0,26.132974,27.0,66.687388,67.0,25.588024,26.0,-32.043333,27.822778,28-09-2015,NaN,NaN,NaN
198,24.897039,25.0,22.131615,22.0,71.119164,71.0,25.267380,25.0,-33.001667,25.161389,08-01-2015,NaN,NaN,NaN


In [55]:
isda_val_df.to_csv("../data/iSDA_features_validation.csv")